In [1]:
from src.time_series_prep import *
import pandas as pd
from models.shared_layer import *
from torchviz import make_dot
from torchsummary import summary
from src.data_preprocessing import *
from src.visualizations import *
from src.global_configs import *
from src.time_series_prep import *
import logging
from src.post_processing import *
from collections import defaultdict
# Initialize logger
logging.basicConfig(
    filename='training.log',  # File where logs will be saved
    level=logging.INFO,  # Set logging level to INFO
    format='%(asctime)s - %(levelname)s - %(message)s',  # Log format
    # filemode='w'  # Overwrite log file on each run
)
logger = logging.getLogger()


In [8]:

scaler = process_all_csv_files(input_folder, output_folder_train, timestamp_col='ts', freq='5min', agg_func='mean')

In [2]:
model_type = 'shared-layer'

prediction_horizons = [3, 6, 9, 12, 15, 18]
abs_patients_errors_PHs =  {key: [] for key in prediction_horizons}
squared_patients_errors_PHs =  {key: [] for key in prediction_horizons}

In [4]:
train_loader, validation_loader, all_train_loader, test_loader, input_shape, input_shape_test, output_shape, output_shape_test =  prepare_data_loader(
                                                                                        window_size,BATCH_SIZE, 6,
                                                                                        model_type, split_ratio = 0.7, df = None, df_test = None, output_folder_train=output_folder_train, shuffle = False)

Shape of X (features): (12, 13612, 12)
Shape of y (targets): (12, 13612)
Shape of X_test (features): (12, 3933, 12)
Shape of y_test (targets): (12, 3933)


In [5]:
model_all_train_read =  SharedLayerModelWithAttention(input_shape =input_shape , output_shape=output_shape)
model_all_train_read.load_state_dict(torch.load(f'saved_models//model_{model_type}_attention_{str(6)}_{0}.pth',weights_only=False))
model_all_train_read.to(device)
model_all_train_read.eval()

SharedLayerModelWithAttention(
  (pre_shared_personalized_tcn): ModuleList(
    (0-11): 12 x Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (relu): ReLU()
        (dropout): Dropout(p=0.2, inplace=False)
        (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (downsample): Conv1d(1, 64, kernel_size=(1,), stride=(1,))
      )
      (1): TemporalBlock(
        (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (relu): ReLU()
        (dropout): Dropout(p=0.2, inplace=False)
        (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
      )
    )
  )
  (pre_shared_personalized_gru): ModuleList(
    (0-11): 12 x GRU(64, 64, batch_first=True)
  )
  (task_specific_attention): ModuleList(
    (0-11): 12 x MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
 

In [ ]:
targets_all_batches

[tensor([[ 1.1304e+00,  7.8496e-01,  2.6535e-01,  5.3330e-01, -2.7867e+00,
           5.0259e-02,  7.6495e-01, -1.0189e+00,  2.6202e-02,  9.6515e-01,
           1.8492e+00,  4.5092e-01],
         [ 1.1501e+00,  8.0193e-01,  2.5280e-01,  5.3330e-01, -2.8802e+00,
           2.8987e-01,  7.2928e-01, -1.1030e+00, -5.7746e-02,  8.8603e-01,
           1.8758e+00,  4.5092e-01],
         [ 1.1697e+00,  8.0193e-01,  2.2752e-01,  5.0552e-01, -2.4780e+00,
           2.0024e-01,  6.9306e-01, -1.1605e+00, -1.4434e-01,  8.0440e-01,
           1.9110e+00,  4.5092e-01],
         [ 1.2085e+00,  7.3352e-01,  2.4019e-01,  4.7742e-01, -2.3536e+00,
           1.0750e-01,  6.5628e-01, -1.2192e+00, -2.3375e-01,  7.0577e-01,
           1.9716e+00,  4.5092e-01],
         [ 1.2372e+00,  6.8131e-01,  2.5280e-01,  4.4898e-01, -2.1555e+00,
          -8.2384e-03,  6.4389e-01, -1.3095e+00, -2.7957e-01,  6.1811e-01,
           2.0140e+00,  4.5092e-01],
         [ 1.2655e+00,  6.6372e-01,  3.0266e-01,  3.7636e-01, -1.

In [ ]:
y_mean = np.mean(targets_all_batches)

In [9]:
outputs_all_batches, targets_all_batches = model_prediction(model_all_train_read,test_loader, device, model_type)

for one_batch in range(len(outputs_all_batches)):
    outputs_np = outputs_all_batches[one_batch].cpu().numpy()
    targets_np = targets_all_batches[one_batch].cpu().numpy()
    for i in range(len(targets_np)): #this prints a list of 12 values of each patient
        for j in range(len(targets_np[i])):
            if targets_np[i][j] != mask_value:
                # outputs_np[i][j] 
                sub_output = np.exp(scaler.inverse_transform([[outputs_np[i][j]]])[0][0])
                sub_target = np.exp(scaler.inverse_transform([[targets_np[i][j]]])[0][0])
                print(sub_output)

181.35971501168618
166.00596489653836
148.03876334641424
161.64390156388436
69.99167309116733
166.51688870101634
183.99933518248628
104.29141252173198
149.57117232838854
200.72797954556125
248.1543624798079
159.402639640675
192.93004415835932
164.32627848004861
148.06730402294795
153.80897354974212
69.14953936846759
162.3927147545169
181.4086096452975
102.1375329357947
150.04901373214892
197.19941368999082
244.69134259549634
159.7493388095517
200.3347022822693
166.12578697014777
147.20749666067437
151.7363875895395
69.15586093060709
158.47979535798126
177.3209769404001
99.60567994185081
148.14026796860156
190.68426788980167
251.67789636259553
160.42363298628766
201.72919365566347
169.1364610581615
148.5555481613814
157.2362854918361
67.76020186541831
157.20700009709262
185.28115069263603
97.86409344056509
144.49092825813912
185.78775875773303
251.8569091152991
158.96948908486922
203.79869771073433
174.34506306284797
151.4235621175329
156.3530228454906
67.84307965161823
152.424450533896

KeyboardInterrupt: 

In [ ]:
y_mean = np.mean(y_true)

# Incrementally compute R²
for true, pred in zip(y_true, y_pred):
    sum_squared_total += (true - y_mean) ** 2
    sum_squared_residual += (true - pred) ** 2
    n += 1  # Update count

# Calculate R²
r2 = 1 - (sum_squared_residual / sum_squared_total)
print(f"R² score: {r2}")

In [ ]:
for prediction_horizon in prediction_horizons:
    train_loader, validation_loader, all_train_loader, test_loader, input_shape, input_shape_test, output_shape, output_shape_test =  prepare_data_loader(
                                                                                            window_size,BATCH_SIZE, prediction_horizon,
                                                                                            model_type, split_ratio = 0.7, df = None, df_test = None, output_folder_train=output_folder_train, shuffle = False)
            
    multiple_runs_each_patient_mae = []
    multiple_runs_each_patient_rmse = []
    for i in range(5):
       
        model_all_train_read =  SharedLayerModelWithAttention(input_shape =input_shape , output_shape=output_shape)
        model_all_train_read.load_state_dict(torch.load(f'saved_models//model_{model_type}_attention_{str(prediction_horizon)}_{i}.pth',weights_only=False))
        model_all_train_read.to(device)
        model_all_train_read.eval()
        each_patient_mae, each_patient_rmse= evaluate_test(model_all_train_read,test_loader, device ,scaler, mask_value, model_type = model_type)

        multiple_runs_each_patient_mae.append(each_patient_mae)
        multiple_runs_each_patient_rmse.append(each_patient_rmse)

    abs_patients_errors_PHs[prediction_horizon] = multiple_runs_each_patient_mae
    squared_patients_errors_PHs[prediction_horizon] = multiple_runs_each_patient_rmse